In [8]:
import whisper
import os
import json


print(f"Whisper version: {whisper.__version__}")

folder_path = os.path.join("Data", "Example 2")
if not os.path.exists(folder_path):
    print("Folder does not exist")

Whisper version: 20230124


# Transcribe the audio

In [9]:
# Audio file
audio_file_name = "eddy_ost.mp3"
audio_file_path = os.path.join(folder_path, audio_file_name)

# Transcribe options (there are more options available, see the documentation)
language = "sv"
beamSize = 5
bestOf = 2

# Download the model
model_name = r"large-v2"
model = whisper.load_model(model_name)


# Different ways to transcribe the audio file
result = model.transcribe(audio_file_path, language="sv")
#result = model.transcribe(audio_file_path, language=language, beam_size=beamSize, best_of=bestOf)
#result = model.transcribe(audio_file_path, **transcribe_options) 
  
print(result["text"])

 Det blev nog en lyckad tillställning. Igår hade några av de gamla gossarna och kamraterna här och......tog oss en liten klarelse. Jag vet inte vad som hände. Det är inte så ofta jag drabbas av den här situationen- och vaknar upp till denna typ av bord och letar runt bland slattarna. Men det var ju ofta under studentåren. Då fann jag ut denna fantastiska rätt. Man tar en ost. Ett halv kilo brukar räcka. Till exempel en mellanlagrad präst. Man tar bort plasten. Det glömmer man ibland. Det är inget gott alls. Man skär bort kanterna. Sen lägger man osten i en glaskål som tål värme- och ställer in den i mikro. Högsta effekt, fem-sex minuter, eller tre-fyra, man får se, tills den har smält. Halvvägs in i smältningen får man röra lite grann i osten- och den blir jämnt smält. Den här behöver nog en minut till. Allt beror på hur effektiv mikrovågsugn man har. Jag får nog vattna chilien innan gorilla kommer hem. Till slut har man en fin smält ostmassa. Man kan jämna till den lite grann. Det här

## Saving the files
The transcribed audio can be saved in a variety of formats. 

In [10]:
#file_name = f"whisper_{whisper.__version__}_{model_name}_beamSize_{beamSize}_bestOf_{bestOf}"
file_name = f"default_whisper_{whisper.__version__}_{model_name}"


# Save to .json ("result" contains a bunch of information)
with open(os.path.join(folder_path, f"{file_name}.json"), "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

# Save to .txt
with open(os.path.join(folder_path, f"{file_name}.txt"), "w", encoding="utf-8") as f:
    f.write(result["text"])

# Save with line breaks as .txt
lines = []
for line in result["segments"]:
    lines.append(line["text"])
with open(os.path.join(folder_path, f"{file_name}_radbryte.txt"), "w", encoding="utf-8") as f:
    for line in lines:
        f.write(f"{line}\n")

In [11]:
# Save as .TSV file
with open(os.path.join(folder_path, f"{file_name}.tsv"), "w", encoding="utf-8") as f:

    print("start", "end", "text", sep="\t", file=f)
    for segment in result["segments"]:
        print(round(1000 * segment['start']), file=f, end="\t")
        print(round(1000 * segment['end']), file=f, end="\t")
        print(segment['text'].strip().replace("\t", " "), file=f, flush=True)

In [12]:
# Save as .SRT file

def format_timestamp(seconds: float, always_include_hours: bool = False, decimal_marker: str = '.'):
    assert seconds >= 0, "non-negative timestamp expected"
    milliseconds = round(seconds * 1000.0)

    hours = milliseconds // 3_600_000
    milliseconds -= hours * 3_600_000

    minutes = milliseconds // 60_000
    milliseconds -= minutes * 60_000

    seconds = milliseconds // 1_000
    milliseconds -= seconds * 1_000

    hours_marker = f"{hours:02d}:" if always_include_hours or hours > 0 else ""
    return f"{hours_marker}{minutes:02d}:{seconds:02d}{decimal_marker}{milliseconds:03d}"

with open(os.path.join(folder_path, f"{file_name}.srt"), "w", encoding="utf-8") as f:
    for i, segment in enumerate(result["segments"], start=1):
        # write srt lines
        print(
            f"{i}\n"
            f"{format_timestamp(segment['start'], always_include_hours=True, decimal_marker='.')} --> "
            f"{format_timestamp(segment['end'], always_include_hours=True, decimal_marker='.')}\n"
            f"{segment['text'].strip().replace('-->', '->')}\n",
            file=f,
            flush=True,
        )

In [13]:
# Save as .VTT file
with open(os.path.join(folder_path, f"{file_name}.vtt"), "w", encoding="utf-8") as f:
    for segment in result["segments"]:
        print(
            f"{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}\n"
            f"{segment['text'].strip().replace('-->', '->')}\n",
            file=f,
            flush=True,
        )